# Time Systems

Simphony has previously only worked with Linear Time-invariant systems, characterized by S-parameter elements. Time-domain Simphony allows a user to create time-variant components, called time-systems. Simphony provides two types of time systems: `SampleModeSystem` and `BlockModeSystem`. These abstract base classes are to be used in Sample Mode simulations and block mode simulations respectively.

# Sample Mode Systems
Sample mode simulations are discrete-time simulations where, for each time step, the output of each component is calculated and used to determine the next set of inputs. This approach enables the simulation of circuits containing components whose inputs depend on the unknown outputs of other components.

In order for this approach to work, each component must be able to maintain state. During Sample Mode Simulations, Simphony looks for two key methods; `init_state` initializes the state of the system, and `step` takes in the current state of the system and a tuple of inputs (one for each port) and returns the next state of the system and a tuple of outputs.

Below is an example of a custom realization of a `SampleModeSystem` which implements a simple phase shifter. We do not enforce that the state variable is of a particular type, so you may use whatever Python object you wish (as long as it does not create issues with the JIT compilation process).

In [3]:
from simphony.time_domain import SampleModeSystem
import jax.numpy as jnp


class PhaseShifter(SampleModeSystem):
    def __init__(
            self,
            voltage_signal
     ) -> None:
        super().__init__()

        self.num_ports = 2
        self.ports = ['o0','o1']
        self.voltage_signal = voltage_signal

    def voltage_to_phase(v):
        """
        NOT A REQUIRED FUNCTION.
        This simply converts the volage signal to a phase shift.
        """
        return 0.5 * v

    def init_state(self):
        """
        REQUIRED FUNCTION
        """
        return jnp.int32(0)

    def step(self, curr_state: jnp.ndarray, input0, input1):
        """
        REQUIRED FUNCTION
        A JIT Compatible function (compatible with lax.scan) that depends on a time-varying voltage.
        """
        v = self.voltage_signal[curr_state]
        phase = self.voltage_to_phase(v)
        out0 = input1 * jnp.exp(1j * phase)
        out1 = input0 * jnp.exp(1j * phase)
        
        return curr_state + 1, (out0, out1)
    


# Jax Compatibility

Since we use "Just In Time" (JIT) compilation methods to run sample mode simulations, there are additional constraints placed on the `step` function, particularly when using loops. For anyone not familiar with the constraints of JIT programming, we recomend you do not create custom sample-mode elements and use one of the models we provide instead.

# Block Mode Simulations

Sample mode simulations are ill-suited for some circuits. For example, you really shouldn't try to simulate a 10-km fiber optic cable by simulating each time step individually, for many reasons. Other techniques exist for these sorts of simulations, such as the split-step Fourier method. Simphony enables these techniques through Block Mode Simulations. 

Because block mode simulations simulate each time system, individually, once (per block), we do not required the `run` method to be compatible with jax or JIT compilation. For example, the following external library has implemented the split-step fourier transform for optical fibers. We can seamlessly integrate this technique into our simulator by using it as a backend for the `run` method.

#

# Sax Compatibility

Since Time-systems will primarily be used in the Time-domain, they are not compatible, by default, with frequency domain solvers like Sax. If desired, you may choose to implement the `frequency_response` method of a time-system so that any circuit that utilizes a time-system is backwards compatible with Sax.